# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
group_data = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": sum})
group_data

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
clean_data = pd.pivot_table(group_data, values = "Quantity", index = "ProductName", columns="CustomerID")
clean_data = clean_data.fillna(0)
clean_data

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(clean_data.T, 'euclidean'))), 
                         index=clean_data.columns, columns=clean_data.columns)
display(distances)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarities_477 = distances[477].sort_values(ascending=False)[1:6].index
print(similarities_477)

Int64Index([3535, 3317, 1072, 1008, 639], dtype='int64', name='CustomerID')


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
top5 = group_data.T[[3535, 3317, 1072, 1008, 639]]
top5 = top5.T
top5

Quantity
CustomerID ProductName                         
3535       Anchovy Paste - 56 G Tube          1
           Apricots - Dried                   1
           Bacardi Breezer - Tropical         1
           Bandage - Fexible 1x3              1
           Beef - Chuck, Boneless             1
...                                         ...
639        Veal - Inside, Choice              1
           Water, Tap                         1
           Wine - Magnotta - Belpaese         1
           Wine - Toasted Head                2
           Wine - Two Oceans Cabernet         1

[272 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
top5_prod = top5.groupby("ProductName").agg({"Quantity":"sum"}).sort_values(by = "Quantity", ascending = False)
top5_prod

,Quantity
ProductName,
Olive - Spread Tapenade,3
Sprouts - Baby Pea Tendrils,3
Wine - Blue Nun Qualitatswein,3
"Veal - Inside, Choice",3
"Pepper - Black, Whole",3
...,...
Flour - Whole Wheat,1
Foam Cup 6 Oz,1
Garlic,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
product_477 = group_data.T[[477]]
product_477 = product_477.T
top_477 = product_477.groupby("ProductName").agg({"Quantity": sum}).sort_values(by = "Quantity", ascending = False)
top_477

,Quantity
ProductName,
Bacardi Breezer - Tropical,3
Cheese - Mix,2
Wine - Charddonnay Errazuriz,2
Soup - Campbells Bean Medley,2
Lettuce - Treviso,2
...,...
Beef - Top Sirloin - Aaa,1
Milk - 1%,1
"Beef - Ground, Extra Lean, Fresh",1


In [11]:
prod_merge = top5_prod.merge(top_477, how='outer', left_on='ProductName', right_on='ProductName')
if [prod_merge["Quantity_y"]==None]:
    prod_merge = pd.DataFrame(prod_merge["Quantity_x"])
prod_merge

,Quantity_x
ProductName,
Olive - Spread Tapenade,3.0
Sprouts - Baby Pea Tendrils,3.0
Wine - Blue Nun Qualitatswein,3.0
"Veal - Inside, Choice",3.0
"Pepper - Black, Whole",3.0
...,...
Isomalt,NaN
"Lamb - Whole, Fresh",NaN
Milk - 1%,NaN


In [12]:
prod_merge.head()

,Quantity_x
ProductName,
Olive - Spread Tapenade,3.0
Sprouts - Baby Pea Tendrils,3.0
Wine - Blue Nun Qualitatswein,3.0
"Veal - Inside, Choice",3.0
"Pepper - Black, Whole",3.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [13]:
recommendation = {}
customers = data["CustomerID"].unique()

In [23]:
for customer in customers:
    product_customer = group_data.T[customer]
    product_customer = product_customer.T
    top_customer = product_customer.groupby("ProductName").agg({"Quantity": "sum"}).sort_values(by = "Quantity", ascending = False)
    top_customer = pd.DataFrame(top_customer)
    
    
    similarities_customers = distances[customer].sort_values(ascending=False)[1:6].index
    all_top5 = group_data.T[similarities_customers]
    all_top5 = all_top5.T
    all_top5_prod = all_top5.groupby("ProductName").agg({"Quantity": "sum"}).sort_values(by = "Quantity", ascending = False)
    all_top5_prod = pd.DataFrame(all_top5_prod)
    
    
    all_prod_merge = all_top5_prod.merge(top_customer, how='outer', left_on='ProductName', right_on='ProductName')
    if [all_prod_merge["Quantity_y"]==None]:
        all_prod_merge = pd.DataFrame(all_prod_merge["Quantity_x"])
        recommendation[customer] = all_prod_merge[:5].index.tolist()
print(recommendation[477])

['Olive - Spread Tapenade', 'Sprouts - Baby Pea Tendrils', 'Wine - Blue Nun Qualitatswein', 'Veal - Inside, Choice', 'Pepper - Black, Whole']


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [24]:
cust_recomm = pd.DataFrame.from_dict(recommendation, orient="index").reset_index()
cust_recomm.columns = ["CustomerID"] + [f"Product {i}" for i in range(1, 6)]
cust_recomm.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Beef - Rib Eye Aaa,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,"Lamb - Pieces, Diced"
1,77352,Wine - Valpolicella Masi,Fuji Apples,Tahini Paste,Ocean Spray - Ruby Red,Tuna - Salad Premix
2,40094,Bread - Italian Corn Meal Poly,"Wine - White, Colubia Cresh",Puree - Mocha,"Cheese - Brie,danish",Cookie - Dough Variety
3,23548,Tea - English Breakfast,Squid - Tubes / Tenticles 10/20,Spinach - Baby,Jolt Cola - Electric Blue,Sprouts - Baby Pea Tendrils
4,78981,Isomalt,Crush - Cream Soda,Vanilla Beans,Cassis,Tea - Earl Grey


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [25]:
distances = pd.DataFrame(1/(1 + squareform(pdist(clean_data.T, 'cosine'))), 
                         index=clean_data.columns, columns=clean_data.columns)
display(distances)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [26]:
recommendation = {}
customers = data["CustomerID"].unique()

In [27]:
for customer in customers:
    product_customer = group_data.T[customer]
    product_customer = product_customer.T
    top_customer = product_customer.groupby("ProductName").agg({"Quantity": "sum"}).sort_values(by = "Quantity", ascending = False)
    top_customer = pd.DataFrame(top_customer)
    
    
    similarities_customers = distances[customer].sort_values(ascending=False)[1:6].index
    all_top5 = group_data.T[similarities_customers]
    all_top5 = all_top5.T
    all_top5_prod = all_top5.groupby("ProductName").agg({"Quantity": "sum"}).sort_values(by = "Quantity", ascending = False)
    all_top5_prod = pd.DataFrame(all_top5_prod)
    
    
    all_prod_merge = all_top5_prod.merge(top_customer, how='outer', left_on='ProductName', right_on='ProductName')
    if [all_prod_merge["Quantity_y"]==None]:
        all_prod_merge = pd.DataFrame(all_prod_merge["Quantity_x"])
        recommendation[customer] = all_prod_merge[:5].index.tolist()
print(recommendation[477])

['Bacardi Breezer - Tropical', 'Halibut - Fletches', 'Chocolate - Feathers', 'Wine - Prosecco Valdobiaddene', 'Pasta - Penne, Rigate, Dry']


In [28]:
cust_recomm = pd.DataFrame.from_dict(recommendation, orient="index").reset_index()
cust_recomm.columns = ["CustomerID"] + [f"Product {i}" for i in range(1, 6)]
cust_recomm.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Longos - Grilled Salmon With Bbq,Jagermeister,Wine - Two Oceans Cabernet,Wine - Fume Blanc Fetzer,Tofu - Firm
1,77352,Cheese - Camembert,Scallops - 10/20,"Chestnuts - Whole,canned",Spice - Peppercorn Melange,Bread - Rye
2,40094,Cumin - Whole,"Beans - Kidney, Red Dry",Juice - V8 Splash,Beef - Inside Round,"Bread - Roll, Soft White Round"
3,23548,Bread - English Muffin,Spinach - Baby,Snapple Lemon Tea,Juice - Orange,Bagel - Plain
4,78981,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Gloves - Goldtouch Disposable,Cheese - Cottage Cheese,Baking Powder
